In [ ]:
import sys 
sys.path.append('..')

import numpy as np
import pandas as pd
import altair as alt
from plotnine.data import mtcars,diamonds

from ModelTools.model.builder.cr_builder import CentralRegBuilder
diamonds.head(10)


In [ ]:
# mod = CentralRegBuilder(cv_method='kfold',cv_split=5,cv_shuffle=True,cv_score='mse').get_model_cv(struct='poly_OLS')
# mod.fit(mtcars.loc[:,['mpg','wt']],mtcars.disp)

In [ ]:
from ModelTools.plot.basic_reg import basic_reg

basic_reg(data=diamonds,x='carat',y='price',formula='y~bs(x,df=3)',ci_level=0.99,reg_method='ols',qr_quantile=0.6)